<a href="https://colab.research.google.com/github/HwangHanJae/recommender_system/blob/main/book_Recommender_Systems/Model_Base_NMF_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NMF(Non-negative Matrix Factorization)
NMF는 음수 미포함 행렬 분해(비음행렬 인수분해)라고 하여 말 그대로 음수를 포함하지 않는 행렬 $X$를 음수를 포함하지 않는 행렬 $W$와 $H$의 곱으로 분해하는 알고리즘입니다.

$$X = WH$$

NMF와 관련하여 참고한 자료는 다음과 같습니다.  
참고링크1 : https://angeloyeo.github.io/2020/10/15/NMF.html  
참고링크2 : https://github.com/ahmadvh/Non-Negative-Matrix-factorization---Implemented-in-python


여기서는 음수를 포함하지 않는 평점을 가진 행렬을 비음평점 행렬 혹은 비음행렬이라고 표현하겠습니다.

비음평점 행렬은 암시적 피드백 데이터세트에 일반적으로 사용됩니다.

암시적 피드백 데이터 세트의 예는 다음과 같습니다.
1. 사용자가 특정 아이템에 대하여 구매 하였을 때 
2. 웹에서 사용자가 특정 아이템에 대해서 클릭 하였을 때
3. 페이스북의 "좋아요" 버튼을 선택하였을 때

위의 예처럼 사용자의 표현은 음수가 아닌 임의의 숫자로 표현할 수 있고, 이러한 숫자는 아이템에 대해서 호감도를 명시하지만 비호감도를 나타내는 것은 아닙니다.

즉 암시적 피드백 데이터 세트는 신뢰를 나타내고 명시적 피드백의 경우는 선호를 나타냅니다.

우리의 목적은 $WH$가 최대한 $X$와 가깝게 만드는 것입니다.
최대한 $X$와 가깝다 라는 의미는 $WH$와 $X$의 차이가 0에 가깝다라는 의미이고 우리가 최소화 해야하는 값입니다.

즉 우리의 목적함수는 프로베니우스 노름(Frobenius norm) $||X-WH||^{2}_{F}$ 정의할 수 있습니다. 

$$min(||X-WH||^{2}_{F})$$

즉 프로베니우스 노름(놈)이 0에 가까워야 하는 것이며 거리(유클리드)가 0에 가깝다는 것입니다.

우리의 목적 함수를 정의하였고 경사하강법을 이용하여 $W$, $H$업데이트 할수 있습니다.

위의 [참고링크1](https://angeloyeo.github.io/2020/10/15/NMF.html)을 보면 $W$, $H$ 행렬을 업데이트하는 과정이 상세하게 잘 나와 있습니다.


결론만 가지고 오면

$$W \Leftarrow W \circ \frac{XH^{T}}{WHH^{T}}$$  
$$H \Leftarrow H \circ \frac{W^{T}X}{W^{T}WH}$$

여기서 $\circ$ 기호는 원소별 곱(Hadamard product)를 의미합니다.

약간 수식을 변경해보겠습니다.

$$X = R$$
$$W = U$$
$$H = V^{T}, H^{T} = V$$

행렬 $U$, $V$의 ($i$, $j$) 번째 항을 각각 $u_{ij}$, $v_{ij}$라고 할 때
식은 아래와 변경됩니다.

$$u_{ij} \Leftarrow \frac{(RV)_{ij}u_{ij}}{(UV^{T}V)_{ij} + \epsilon}$$

$$v_{ij} \Leftarrow \frac{(R^{T}U)_{ij}v_{ij}}{(VU^{T}U)_{ij} + \epsilon}$$

여기서 $\epsilon \approx 10^{-9}$와 같은 작은 값으로 분모가 0이 되지 않게 하여 안정성을 높입니다. $U$, $V$가 업데이트 될때 모든 항목이 "동시에" 업데이트가 됩니다.  
그리고 초기화 단계에서는 (0, 1)의 임의의 값으로 초기화 됩니다.

NMF 또한 다른 행렬분해와 마찬가지로 정규화를 사용할 수 있습니다.   
목적함수에 $\frac{\lambda_{1}||U||^{2}}{2}+\frac{\lambda_{2}||V||^{2}}{2}$ 패널티를 추가하여 만들수 있습니다. 여기서 $\lambda_{1} > 0$ 및 $\lambda_{2} > 0$은 정규화 파라미터입니다.

식은 아래와 같이 업데이트 됩니다.

$$u_{ij} \Leftarrow max\left\{\left[\frac{(RV)_{ij}-\lambda_{1}u_{ij}}{(UV^{T}V)_{ij} + \epsilon}\right]u_{ij},0\right\}$$

$$v_{ij} \Leftarrow max\left\{\left[\frac{(R^{T}U)_{ij}-\lambda_{2}v_{ij}}{(VU^{T}U)_{ij} + \epsilon}\right]v_{ij},0\right\}$$
